# Командная часть проекта

In [102]:
data_config = {
    'spindalis' : {
    },
    'japonicum' : {
    },
    'bovis' : {
    },
    'mattheei' : {
    },
    'haematobium' : {
    },
    'mansoni' : {
    },
    'intercalatum': {
        'epigenetic_file' : 'epigenetic_genes.csv',
        'g4_file' : 'g4_segments.csv',
        'zrnk_file' : 'zrnk_segments.csv',
        'gff_url' : 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/944/470/385/GCA_944470385.2_tdSchInte2.2/GCA_944470385.2_tdSchInte2.2_genomic.gff.gz',
        'fna_url' : 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/944/470/385/GCA_944470385.2_tdSchInte2.2/GCA_944470385.2_tdSchInte2.2_genomic.fna.gz'
    },
    'mekongi' : {
    },
}

In [117]:
# скачаем аннотации

# скачивание файлов генома и аннотации

import requests
import logging
import pathlib
import glob
import os
import sys
import itertools

logging.basicConfig(level=logging.INFO, force = True)

def download_file(url: str, local_file_name: pathlib.Path):
    if pathlib.Path(local_file_name).is_file():
        logging.info(f'file {local_file_name} already exists')
        return local_file_name
    logging.info(f'downloading {local_file_name}')
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_file_name, 'wb') as out_file:
            for chunk in r.iter_content(chunk_size=8192): 
                out_file.write(chunk)

    return local_file_name

gff_files = dict()
fna_files = dict()

# скачиваем аннотации
for current_organism, current_data_config in data_config.items():
    if 'gff_url' in current_data_config:
        gff_files[current_organism] = download_file(current_data_config['gff_url'], f'data/{current_data_config['gff_url'].split('/')[-1]}')
    else:
        logging.warning(f'No gff url provided for {current_organism}')
    
    if 'fna_url' in current_data_config:
        fna_files[current_organism] = download_file(current_data_config['fna_url'], f'data/{current_data_config['fna_url'].split('/')[-1]}')
    else:
        logging.warning(f'No fna url provided for {current_organism}')

print(file=sys.stderr)

# распакуем сжатые файлы для удобства
for organism, filepath in itertools.chain(gff_files.items(), fna_files.items()):
    current_file_path = pathlib.Path(filepath)
    assert current_file_path.is_file()
    if current_file_path.suffix == '.gz':
        current_extracted_file = current_file_path.parent / pathlib.Path(filepath).stem
        if current_extracted_file.is_file():
            logging.info(f'{current_extracted_file} already exists')
        else:
            logging.info(f'extracting {current_extracted_file}')
            os.system(f'gunzip -k {current_extracted_file}')

for organism, filepath in gff_files.items():
    if pathlib.Path(filepath).suffix == '.gz':
        gff_files[organism] = current_file_path.parent / pathlib.Path(filepath).stem

for organism, filepath in fna_files.items():
    if pathlib.Path(filepath).suffix == '.gz':
        fna_files[organism] = current_file_path.parent / pathlib.Path(filepath).stem


INFO:root:file data/GCA_944470385.2_tdSchInte2.2_genomic.gff.gz already exists
INFO:root:file data/GCA_944470385.2_tdSchInte2.2_genomic.fna.gz already exists

INFO:root:data/GCA_944470385.2_tdSchInte2.2_genomic.gff already exists
INFO:root:data/GCA_944470385.2_tdSchInte2.2_genomic.fna already exists


In [106]:
# функции для чтения данных из результатов индивидуальных проектов

import pandas as pd
import pathlib

def get_epigenetic_genes(organism: str):
    assert organism in data_config, f'unknown organism: {organism}'

    if 'epigenetic_file' in data_config[organism]:
        file_name = data_config[organism]['epigenetic_file']
        if pathlib.Path(file_name).suffix == '.csv':
            res = pd.read_csv(f'data_in/{organism}/{data_config[organism]['epigenetic_file']}')
            res = res[['Gene_ID']]
            res.columns = ['gene']
            return res
        else:
            assert False, "Please, add support for your file type" # TODO
    else:
        logging.warning(f'No epigenetic_file specified for {organism}')
        return None


### 1. От каждого организма в группе Выбрать из генов отвечающих за эпигенетику 4-5 (как минимум 1 на члена группы) и построить для них выравнивания и деревья.

In [119]:
import logging

epigenetic_genes_for_alignment = []

for organism in data_config:
    current_epigenetic_genes = get_epigenetic_genes(organism)
    if current_epigenetic_genes is not None:
        current_epigenetic_genes_selected = current_epigenetic_genes.sample(5, random_state=42)
        assert len(current_epigenetic_genes_selected) == 5, 'too small current_epigenetic_genes_selected size'
        logging.info(f'Adding genes for {organism}')
        current_epigenetic_genes_selected['organism'] = organism
        epigenetic_genes_for_alignment.append(current_epigenetic_genes_selected)
    else:
        logging.warning(f'current_epigenetic_genes({organism}) is None')

epigenetic_genes_for_alignment = pd.concat(epigenetic_genes_for_alignment)
epigenetic_genes_for_alignment

INFO:root:Adding genes for intercalatum


,gene,organism
218,gene-SINT2_LOCUS1809,intercalatum
66,gene-SINT2_LOCUS5354,intercalatum
9,gene-SINT2_LOCUS3201,intercalatum
170,gene-SINT2_LOCUS4930,intercalatum
15,gene-SINT2_LOCUS5557,intercalatum


In [120]:
# Добавим в табличку хромосому координаты каждого

import logging

gene_organism_to_position = dict()

for _, row in epigenetic_genes_for_alignment.iterrows():
    gene_organism_to_position[(row['organism'], row['gene'])] = None

for organism in data_config:
    if organism in gff_files:
        with open(gff_files[organism], 'r') as gff_file:
            for row in gff_file:
                current_row = row.strip()
                if len(current_row) == 0 or current_row[0] == '#':
                    continue
                current_row_splitted = row.split()

                if current_row_splitted[2] == 'gene':
                    current_gene_id = current_row_splitted[8].split('ID=')[1].split(';')[0]
                    if (organism, current_gene_id) in gene_organism_to_position:
                        gene_organism_to_position[(organism, current_gene_id)] = (current_row_splitted[0], int(current_row_splitted[3]), int(current_row_splitted[4]))
    else:
        logging.warning(f'No such gff file for {organism}')

def get_position(row):
    res = gene_organism_to_position[(row['organism'], row['gene'])]
    return res

epigenetic_genes_for_alignment[['record_id', 'start', 'end']] = \
    epigenetic_genes_for_alignment.apply(get_position, axis=1, result_type='expand')

epigenetic_genes_for_alignment

,gene,organism,record_id,start,end
218,gene-SINT2_LOCUS1809,intercalatum,OX103876.1,69497234,69537522
66,gene-SINT2_LOCUS5354,intercalatum,OX103879.1,30466976,30476049
9,gene-SINT2_LOCUS3201,intercalatum,OX103877.1,38092074,38115003
170,gene-SINT2_LOCUS4930,intercalatum,OX103879.1,12765221,12771890
15,gene-SINT2_LOCUS5557,intercalatum,OX103879.1,37810964,37839258


In [131]:
# Добавим в табличку последовательность гена

import pandas as pd
import logging
from Bio import SeqIO

record_to_position = dict()

for _, row in epigenetic_genes_for_alignment.iterrows():
    if row['record_id'] not in record_to_position:
        record_to_position[row['record_id']] = dict()
    record_to_position[row['record_id']][(row['start'], row['end'])] = None

record_to_position

for organism in data_config:
    if organism in faa_files:
        records = list(SeqIO.parse(fna_files[organism], "fasta"))
        for record in records:
            if record.id in record_to_position:
                for requested_range in record_to_position[record.id]:
                    record_to_position[record.id][requested_range] = record.seq[requested_range[0]-1:requested_range[1]]

    else:
        logging.warning(f'No such fna file for {organism}')

def get_sequence(row):
    return [record_to_position[row['record_id']][(row['start'], row['end'])]]
        
epigenetic_genes_for_alignment[['data']] = \
    epigenetic_genes_for_alignment.apply(get_sequence, axis=1, result_type='expand')

epigenetic_genes_for_alignment['data'] = epigenetic_genes_for_alignment['data'].apply(lambda s : str(s).lower())
epigenetic_genes_for_alignment

,gene,organism,record_id,start,end,data
218,gene-SINT2_LOCUS1809,intercalatum,OX103876.1,69497234,69537522,atggatgtatgctttatttcatttaacttctaagatgcacttcatt...
66,gene-SINT2_LOCUS5354,intercalatum,OX103879.1,30466976,30476049,acagaataaaagaggtgtactgttaacaacagccagagtgcttttt...
9,gene-SINT2_LOCUS3201,intercalatum,OX103877.1,38092074,38115003,tctggttgtactggcggatgacttagaggtaggattttgttttcct...
170,gene-SINT2_LOCUS4930,intercalatum,OX103879.1,12765221,12771890,tcataaatgtaaccgtttagcacatgtaggtggtaatgacggtgat...
15,gene-SINT2_LOCUS5557,intercalatum,OX103879.1,37810964,37839258,gtttaagaagttgcgccttcttcagatcttagatggtattaacatt...


In [ ]:
# TODO